# **Quy trình tiền xử lý ảnh**

Đoạn mã này thực hiện tiền xử lý ảnh, chuyển đổi ảnh đầu vào thành:
1. **Ảnh xám (Grayscale)**: Biểu diễn cường độ sáng của ảnh.
2. **Ảnh nhị phân (Binary)**: Làm nổi bật các đặc trưng quan trọng.

## **Các bước trong quy trình tiền xử lý**

### 1. **Chuyển ảnh sang ảnh xám**
- Hàm `extractValue` chuyển đổi ảnh từ không gian màu BGR sang không gian màu HSV, sau đó lấy kênh **Value**.
- Kênh Value đại diện cho cường độ sáng, giúp loại bỏ nhiễu từ thông tin màu sắc.

### 2. **Tăng cường độ tương phản**
- Hàm `maximizeContrast` sử dụng các phép biến đổi hình thái học:
  - **Top-Hat**: Làm nổi bật chi tiết sáng trên nền tối.
  - **Black-Hat**: Làm nổi bật chi tiết tối trên nền sáng.
- Kết hợp ảnh gốc với kết quả của các phép biến đổi để tăng cường độ tương phản.

### 3. **Làm mờ bằng Gaussian Blur**
- Sử dụng bộ lọc Gaussian với kernel kích thước (5, 5) để giảm nhiễu, làm ảnh mượt hơn.

### 4. **Tạo ảnh nhị phân**
- Sử dụng ngưỡng thích ứng (**adaptive thresholding**) để tạo ảnh nhị phân:
  - Phương pháp này tính toán ngưỡng cục bộ cho từng vùng nhỏ, xử lý tốt ảnh có độ sáng không đồng đều.
  - Các vùng quan trọng trong ảnh được tách biệt rõ ràng.

## **Kết quả đầu ra**
- **`imgGrayscale`**: Ảnh xám của ảnh đầu vào.
- **`imgThresh`**: Ảnh nhị phân, dùng để phát hiện đặc trưng hoặc xử lý tiếp theo.



In [ ]:
import cv2
import numpy as np
import math

def preprocess(imgOriginal): #tham số truyền vào là ảnh gốc
    imgGrayscale = extractValue(imgOriginal) # LẤY ẢNH ĐEN TRẮNG
    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale) 
    height, width = imgGrayscale.shape
    imgBlurred = np.zeros((height, width, 1), np.uint8) # tạo np chứa
    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, (5,5), 0)
    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 9)

    #Tạo ảnh nhị phân
    return imgGrayscale, imgThresh
    #Trả về ảnh xám và ảnh nhị phân



def extractValue(imgOriginal): # HÀM CHO RA ẢNH ĐEN TRẮNG
    height, width, numChannels = imgOriginal.shape
                  # numChannels: bộ màu sử dụng (red-green blue: 3, bw:1)
        
    imgHSV = np.zeros((height, width, 3), np.uint8)
    imgHSV = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2HSV)

    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)
    
    #màu sắc, độ bão hòa, giá trị cường độ sáng
    #Không chọn màu RBG vì vd ảnh màu đỏ sẽ còn lẫn các màu khác nữa nên khó xđ ra "một màu" 
    
    return imgValue # ẢNH VALUE SẼ LÀ ẢNH ĐEN TRẮNG
# end function


def maximizeContrast(imgGrayscale):
    #Làm cho độ tương phản lớn nhất 
    height, width = imgGrayscale.shape
    
    imgTopHat = np.zeros((height, width, 1), np.uint8)
    
    imgBlackHat = np.zeros((height, width, 1), np.uint8)
    
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)) #tạo bộ lọc kernel
    
    imgTopHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_TOPHAT, structuringElement, iterations = 10) #nổi bật chi tiết sáng trong nền tối
    #cv2.imwrite("tophat.jpg",imgTopHat)
    imgBlackHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_BLACKHAT, structuringElement, iterations = 10) #Nổi bật chi tiết tối trong nền sáng
    #cv2.imwrite("blackhat.jpg",imgBlackHat)
    imgGrayscalePlusTopHat = cv2.add(imgGrayscale, imgTopHat) 
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    #Kết quả cuối là ảnh đã tăng độ tương phản 
    return imgGrayscalePlusTopHatMinusBlackHat
# end function



# **Chương trình huấn luyện nhận dạng ký tự từ ảnh**

Đoạn mã này thực hiện việc huấn luyện một mô hình nhận dạng ký tự dựa trên một tập ảnh ký tự đầu vào (**training_chars.png**). Chương trình bao gồm các bước sau:

---

## **1. Đọc và tiền xử lý ảnh huấn luyện**
- Ảnh huấn luyện chứa các ký tự cần nhận dạng.
- Tiền xử lý ảnh gồm các bước:
  - Chuyển ảnh sang ảnh xám (**Grayscale**).
  - Làm mờ ảnh bằng **Gaussian Blur** để giảm nhiễu.
  - Phân ngưỡng ảnh (**Adaptive Thresholding**) để tạo ảnh nhị phân, làm nổi bật các ký tự.

---

## **2. Phát hiện và xử lý đường viền (Contours)**
- Sử dụng hàm `cv2.findContours` để phát hiện tất cả các vùng ký tự (đường viền) trên ảnh nhị phân.
- Chỉ giữ lại những vùng có diện tích lớn hơn **MIN_CONTOUR_AREA** (để loại bỏ nhiễu).

---

## **3. Cắt và chuẩn hóa từng ký tự**
- Với mỗi vùng ký tự tìm được:
  - Vẽ hình chữ nhật bao quanh ký tự trên ảnh gốc.
  - Cắt ảnh nhị phân của ký tự ra khỏi ảnh huấn luyện.
  - Chuẩn hóa kích thước ký tự thành kích thước cố định (20x30 pixel) để đảm bảo tính nhất quán.

---

## **4. Gán nhãn thủ công**
- Hiển thị từng ký tự đã chuẩn hóa để người dùng gán nhãn (nhập từ bàn phím).
- Chỉ cho phép các ký tự hợp lệ (0-9, A-Z) dựa trên danh sách `intValidChars`.
- Nếu nhấn phím **ESC**, chương trình sẽ thoát.

---

## **5. Lưu dữ liệu huấn luyện**
- Mỗi ký tự được chuẩn hóa sẽ được lưu dưới dạng:
  - **Vector phẳng (Flattened)**: Biểu diễn toàn bộ pixel của ký tự thành một mảng 1 chiều.
  - **Nhãn (Classification)**: Ký tự mà người dùng gán nhãn.
- Tất cả dữ liệu được lưu vào hai file:
  - **`classifications.txt`**: Lưu nhãn của các ký tự.
  - **`flattened_images.txt`**: Lưu các vector phẳng của ảnh ký tự.

---

## **Mục tiêu**
- Chuẩn bị dữ liệu huấn luyện (ảnh + nhãn) để sử dụng trong các mô hình nhận dạng ký tự, như OCR (Optical Character Recognition).


In [ ]:
import numpy as np
import cv2
import sys

MIN_CONTOUR_AREA = 40

# kích cỡ mặc định của biển số xe
RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

def main():
    imgTrainingNumbers = cv2.imread("training_chars.png")            # training_chars là ảnh chứa tập ký tự
    
    imgGray = cv2.cvtColor(imgTrainingNumbers, cv2.COLOR_BGR2GRAY)          
    imgBlurred = cv2.GaussianBlur(imgGray, (5,5), 0)                    

                                                        
    imgThresh = cv2.adaptiveThreshold(imgBlurred,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)

    cv2.imshow("imgThresh", imgThresh)      

    
    
    imgThreshCopy = imgThresh.copy()    
    npaContours, hierarchy = cv2.findContours(imgThreshCopy,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)       
    npaFlattenedImages =  np.empty((0, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
   

    intClassifications = []         # declare empty classifications list, this will be our list of how we are classifying our chars from user input, we will write to file at the end

                                    # possible chars we are interested in are digits 0 through 9, put these in list intValidChars
    intValidChars = [ord('0'), ord('1'), ord('2'), ord('3'), ord('4'), ord('5'), ord('6'), ord('7'), ord('8'), ord('9'),
                     ord('A'), ord('B'), ord('C'), ord('D'), ord('E'), ord('F'), ord('G'), ord('H'), ord('I'), ord('J'),
                     ord('K'), ord('L'), ord('M'), ord('N'), ord('O'), ord('P'), ord('Q'), ord('R'), ord('S'), ord('T'),
                     ord('U'), ord('V'), ord('W'), ord('X'), ord('Y'), ord('Z')] #Là mã ascii của mấy chữ này

    for npaContour in npaContours:                          # for each contour
        if cv2.contourArea(npaContour) > MIN_CONTOUR_AREA:          # if contour is big enough to consider
            [intX, intY, intW, intH] = cv2.boundingRect(npaContour)         # get and break out bounding rect

                                                # draw rectangle around each contour as we ask user for input
            cv2.rectangle(imgTrainingNumbers,           # draw rectangle on original training image
                          (intX, intY),                 # upper left corner
                          (intX+intW,intY+intH),        # lower right corner
                          (0, 0, 255),                  # red
                          2)                            # thickness

            imgROI = imgThresh[intY:intY+intH, intX:intX+intW]                                  # crop char out of threshold image
            imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))     # resize image, this will be more consistent for recognition and storage

            cv2.imshow("imgROI", imgROI)                    # show cropped out char for reference
            cv2.imshow("imgROIResized", imgROIResized)      # show resized image for reference
            
            cv2.imshow("training_numbers.png", imgTrainingNumbers)      # show training numbers image, this will now have red rectangles drawn on it

            intChar = cv2.waitKey(0)                     # get key press

            if intChar == 27:                   # if esc key was pressed
                sys.exit()                      # exit program
            elif intChar in intValidChars:      # else if the char is in the list of chars we are looking for . . .

                intClassifications.append(intChar)        # append classification char to integer list of chars (we will convert to float later before writing to file)
                #Là file chứa label của tất cả các ảnh mẫu, tổng cộng có 32 x 5 = 160 mẫu.
                npaFlattenedImage = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))  # flatten image to 1d numpy array so we can write to file later
                
                npaFlattenedImages = np.append(npaFlattenedImages, npaFlattenedImage, 0)                    # add current flattened impage numpy array to list of flattened image numpy arrays
           
        
    fltClassifications = np.array(intClassifications, np.float32)                   # convert classifications list of ints to numpy array of floats
    
    npaClassifications = fltClassifications.reshape((fltClassifications.size, 1))   # flatten numpy array of floats to 1d so we can write to file later

    print ("\n\ntraining complete !!\n")

    np.savetxt("classifications.txt", npaClassifications)           # write flattened images to file
    np.savetxt("flattened_images.txt", npaFlattenedImages)          #

    cv2.destroyAllWindows()             # remove windows from memory

    return

if __name__ == "__main__":
    main()


# # **Nhận diện biển số xe từ hình ảnh**

Đoạn mã này thực hiện nhận diện biển số xe từ hình ảnh đầu vào. Quá trình bao gồm các bước chính sau:

---

## **1. Chuẩn bị dữ liệu**
- Đọc ảnh biển số xe từ đường dẫn `data/image/10.jpg` và thay đổi kích thước thành 1920x1080.
- Tải các dữ liệu huấn luyện (`classifications.txt` và `flattened_images.txt`) để sử dụng với thuật toán **KNN**.
- Tạo một đối tượng KNN (`cv2.ml.KNearest_create()`) và huấn luyện trên tập dữ liệu đã nạp.

---

## **2. Tiền xử lý ảnh**
- Tiền xử lý ảnh đầu vào:
  - Sử dụng hàm `Preprocess.preprocess` để chuyển ảnh sang dạng xám và nhị phân.
  - Áp dụng **Canny Edge Detection** để tìm biên.
  - Thực hiện **dilation** (giãn nở) để làm rõ các vùng biên số xe.

---

## **3. Phát hiện biển số**
- Tìm tất cả các **contour** (đường viền) từ ảnh nhị phân giãn nở.
- Lọc ra 10 contour lớn nhất và kiểm tra:
  - Contour có 4 cạnh (xấp xỉ đa giác) sẽ được coi là vùng biển số khả thi.
  - Vẽ khung xung quanh các vùng biển số được phát hiện.

---

## **4. Xoay và cắt biển số**
- Tính góc nghiêng của biển số bằng tọa độ các đỉnh của contour.
- Sử dụng phép quay (rotation) để xoay vùng biển số về vị trí thẳng đứng.
- Cắt vùng biển số và thay đổi kích thước để chuẩn hóa (phóng đại x3 để tăng độ chi tiết).

---

## **5. Xử lý các ký tự trên biển số**
### **a. Phân đoạn ký tự**
- Áp dụng **morphological operations** (dilate) để làm rõ các ký tự.
- Tìm tất cả các **contour** đại diện cho ký tự.
- Lọc các ký tự dựa trên:
  - Tỷ lệ diện tích ký tự so với vùng biển số.
  - Tỷ lệ chiều rộng và chiều cao ký tự.

### **b. Nhận diện ký tự**
- Cắt từng ký tự ra khỏi biển số và chuẩn hóa kích thước (20x30 pixel).
- Sử dụng KNN để dự đoán ký tự từ mô hình đã huấn luyện.
- Phân loại ký tự thành **dòng trên** hoặc **dòng dưới** của biển số tùy thuộc vào vị trí.

---

## **6. Xuất kết quả**
- Hiển thị biển số nhận diện được dưới dạng chuỗi ký tự (`first_line - second_line`).
- Vẽ các ký tự được nhận diện lên hình ảnh biển số và hiển thị kết quả cuối cùng.

---

## **7. Mục tiêu**
- Xây dựng một hệ thống nhận diện biển số xe tự động từ hình ảnh, phục vụ các ứng dụng như giám sát giao thông, bãi đỗ xe thông minh.


In [ ]:
import math

import cv2
import numpy as np
import nbimporter

import Preprocess



n = 1

Min_char = 0.01
Max_char = 0.09

RESIZED_IMAGE_WIDTH = 20
RESIZED_IMAGE_HEIGHT = 30

img = cv2.imread("data/image/10.jpg")
img = cv2.resize(img, dsize=(1920, 1080))

npaClassifications = np.loadtxt("classifications.txt", np.float32)
npaFlattenedImages = np.loadtxt("flattened_images.txt", np.float32)
npaClassifications = npaClassifications.reshape(
    (npaClassifications.size, 1))  # reshape numpy array to 1d, necessary to pass to call to train
kNearest = cv2.ml.KNearest_create()  # instantiate KNN object
kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)
#########################

################ Image Preprocessing #################

imgGrayscaleplate, imgThreshplate = Preprocess.preprocess(img)


canny_image = cv2.Canny(imgThreshplate, 250, 255)  # Canny Edge
kernel = np.ones((3, 3), np.uint8)
dilated_image = cv2.dilate(canny_image, kernel, iterations=1)  # Dilation
# cv2.imshow("dilated_image",dilated_image)

###########################################

###### Draw contour and filter out the license plate  #############
contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]  # Lấy 10 contours có diện tích lớn nhất
# cv2.drawContours(img, contours, -1, (255, 0, 255), 3) # Vẽ tất cả các ctour trong hình lớn

screenCnt = []
for c in contours:
    peri = cv2.arcLength(c, True)  # Tính chu vi
    approx = cv2.approxPolyDP(c, 0.06 * peri, True)  # làm xấp xỉ đa giác, chỉ giữ contour có 4 cạnh
    [x, y, w, h] = cv2.boundingRect(approx.copy())
    ratio = w / h
    # cv2.putText(img, str(len(approx.copy())), (x,y),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 3)
    # cv2.putText(img, str(ratio), (x,y),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 3)
    if (len(approx) == 4):
        screenCnt.append(approx)

        cv2.putText(img, str(len(approx.copy())), (x, y), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 0), 3)

if screenCnt is None:
    detected = 0
    print("No plate detected")
else:
    detected = 1

if detected == 1:

    for screenCnt in screenCnt:
        cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)  # Khoanh vùng biển số xe

        ############## Find the angle of the license plate #####################
        (x1, y1) = screenCnt[0, 0]
        (x2, y2) = screenCnt[1, 0]
        (x3, y3) = screenCnt[2, 0]
        (x4, y4) = screenCnt[3, 0]
        array = [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        sorted_array = array.sort(reverse=True, key=lambda x: x[1])
        (x1, y1) = array[0]
        (x2, y2) = array[1]
        doi = abs(y1 - y2)
        ke = abs(x1 - x2)
        angle = math.atan(doi / ke) * (180.0 / math.pi)

        ####################################

        ########## Crop out the license plate and align it to the right angle ################

        mask = np.zeros(imgGrayscaleplate.shape, np.uint8)
        new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1, )
        # cv2.imshow("new_image",new_image)

        # Cropping
        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))

        roi = img[topx:bottomx, topy:bottomy]
        imgThresh = imgThreshplate[topx:bottomx, topy:bottomy]
        ptPlateCenter = (bottomx - topx) / 2, (bottomy - topy) / 2

        if x1 < x2:
            rotationMatrix = cv2.getRotationMatrix2D(ptPlateCenter, -angle, 1.0)
        else:
            rotationMatrix = cv2.getRotationMatrix2D(ptPlateCenter, angle, 1.0)

        roi = cv2.warpAffine(roi, rotationMatrix, (bottomy - topy, bottomx - topx))
        imgThresh = cv2.warpAffine(imgThresh, rotationMatrix, (bottomy - topy, bottomx - topx))
        roi = cv2.resize(roi, (0, 0), fx=3, fy=3)
        imgThresh = cv2.resize(imgThresh, (0, 0), fx=3, fy=3)

        ####################################

        #################### Prepocessing and Character segmentation ####################
        kerel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thre_mor = cv2.morphologyEx(imgThresh, cv2.MORPH_DILATE, kerel3)
        cont, hier = cv2.findContours(thre_mor, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cv2.imshow(str(n + 20), thre_mor)
        cv2.drawContours(roi, cont, -1, (100, 255, 255), 2)  # Vẽ contour các kí tự trong biển số

        ##################### Filter out characters #################
        char_x_ind = {}
        char_x = []
        height, width, _ = roi.shape
        roiarea = height * width

        for ind, cnt in enumerate(cont):
            (x, y, w, h) = cv2.boundingRect(cont[ind])
            ratiochar = w / h
            char_area = w * h
            # cv2.putText(roi, str(char_area), (x, y+20),cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 2)
            # cv2.putText(roi, str(ratiochar), (x, y+20),cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 2)

            if (Min_char * roiarea < char_area < Max_char * roiarea) and (0.25 < ratiochar < 0.7):
                if x in char_x:  # Sử dụng để dù cho trùng x vẫn vẽ được
                    x = x + 1
                char_x.append(x)
                char_x_ind[x] = ind

                # cv2.putText(roi, str(char_area), (x, y+20),cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 2)

        ############ Character recognition ##########################

        char_x = sorted(char_x)
        strFinalString = ""
        first_line = ""
        second_line = ""

        for i in char_x:
            (x, y, w, h) = cv2.boundingRect(cont[char_x_ind[i]])
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

            imgROI = thre_mor[y:y + h, x:x + w]  # Crop the characters

            imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))  # resize image
            npaROIResized = imgROIResized.reshape(
                (1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))

            npaROIResized = np.float32(npaROIResized)
            _, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized,k=3)  # call KNN function find_nearest;
            strCurrentChar = str(chr(int(npaResults[0][0])))  # ASCII of characters
            cv2.putText(roi, strCurrentChar, (x, y + 50), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 3)

            if (y < height / 3):  # decide 1 or 2-line license plate
                first_line = first_line + strCurrentChar
            else:
                second_line = second_line + strCurrentChar

        print("\n License Plate " + str(n) + " is: " + first_line + " - " + second_line + "\n")
        roi = cv2.resize(roi, None, fx=0.75, fy=0.75)
        cv2.imshow(str(n), cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))

        # cv2.putText(img, first_line + "-" + second_line ,(topy ,topx),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 2)
        n = n + 1

img = cv2.resize(img, None, fx=0.5, fy=0.5)
cv2.imshow('License plate', img)

cv2.waitKey(0)

## **KNN trong nhận diện biển số**

**KNN (K-Nearest Neighbors)** là một thuật toán học máy thuộc nhóm **không giám sát**, sử dụng để phân loại dữ liệu dựa trên khoảng cách đến các điểm lân cận. Trong bài toán nhận diện biển số này, KNN được sử dụng để phân loại các ký tự từ ảnh biển số. Cụ thể:

---

### **1. Huấn luyện mô hình KNN**
- **Dữ liệu huấn luyện**:
  - Tập dữ liệu huấn luyện bao gồm:
    - **classifications.txt**: Chứa nhãn (label) của các ký tự (ví dụ: 'A', 'B', '1', '2',...).
    - **flattened_images.txt**: Chứa các ảnh ký tự đã được xử lý và chuyển thành mảng 1D (vector).
  - Mỗi ảnh ký tự được chuẩn hóa về kích thước **20x30 pixel**, sau đó được "phẳng hóa" thành một mảng gồm \( 20 \times 30 = 600 \) phần tử.

- **Quá trình huấn luyện**:
  ```python
  kNearest = cv2.ml.KNearest_create()  # Tạo đối tượng KNN
  kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)


## **2. Dự đoán ký tự từ biển số**

### **2.1. Cắt và chuẩn hóa ký tự**
- Sau khi phát hiện biển số và khoanh vùng các ký tự, từng ký tự sẽ được xử lý:
  - **Chuyển kích thước** về chuẩn \( 20 \times 30 \) pixel, đảm bảo kích thước đầu vào phù hợp với mô hình:
    ```python
    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
    ```
  - **Chuyển đổi sang mảng 1D** để đưa vào mô hình KNN:
    ```python
    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
    npaROIResized = np.float32(npaROIResized)
    ```

### **2.2. Nhận diện ký tự với KNN**
- **Dự đoán ký tự** sử dụng mô hình KNN đã được huấn luyện:
  ```python
  _, npaResults, _, _ = kNearest.findNearest(npaROIResized, k=3)
  strCurrentChar = str(chr(int(npaResults[0][0])))


## **3. Gắn nhãn ký tự và xuất kết quả**

### **3.1. Gắn nhãn ký tự lên ảnh biển số**
- Sau khi nhận diện từng ký tự, chúng ta sẽ thực hiện việc **gắn nhãn** lên ảnh biển số:
  - **Vẽ hình chữ nhật** xung quanh từng ký tự để làm rõ vùng chứa ký tự đó:
    ```python
    cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)
    ```
    - Trong đó, `x`, `y`, `w`, `h` là tọa độ và kích thước của hình chữ nhật bao quanh ký tự.
  
  - **Gắn ký tự nhận diện được** lên ảnh, giúp người dùng dễ dàng nhìn thấy kết quả nhận diện:
    ```python
    cv2.putText(roi, strCurrentChar, (x, y + 50), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 255, 0), 3)
    ```

### **3.2. Phân chia ký tự thành các dòng**
- Biển số xe có thể có **1 hoặc 2 dòng**. Để phân chia các ký tự vào từng dòng, chúng ta sử dụng thông tin tọa độ \(y\) của mỗi ký tự:
  - Nếu \( y \) của ký tự nằm ở phần trên của ảnh (dưới 1/3 chiều cao ảnh), ký tự đó sẽ được thêm vào dòng đầu tiên (`first_line`).
  - Nếu \( y \) của ký tự nằm ở phần dưới, ký tự đó sẽ được thêm vào dòng thứ hai (`second_line`).
    ```python
    if y < height / 3:
        first_line += strCurrentChar
    else:
        second_line += strCurrentChar
    ```

### **3.3. Xuất kết quả nhận diện**
- Sau khi phân chia và nhận diện tất cả các ký tự, chúng ta sẽ **in kết quả biển số** dưới dạng chuỗi:
  ```python
  print("\n License Plate " + str(n) + " is: " + first_line + " - " + second_line + "\n")
